# Read Data

In [7]:
import os
import pickle

from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader
from llama_parse import LlamaParse
import nest_asyncio

load_dotenv()
nest_asyncio.apply()

In [12]:
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")

# Do not parse twice. Check if the file is already parsed
documents = None
if not os.path.exists("./parsed-objects/documents.pickle"):
    print("Parsing documents")
    parser = LlamaParse(
        api_key=LLAMA_CLOUD_API_KEY,
        result_type="text",
        verbose=True,
    )

    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(
        "./data", file_extractor=file_extractor
    ).load_data()
    with open("./parsed-objects/documents.pickle", "wb") as f:
        pickle.dump(documents, f)
else:
    print("Loading parsed documents from pickle")
    with open("./parsed-objects/documents.pickle", "rb") as f:
        documents = pickle.load(f)

assert documents is not None, "Documents are not loaded"

Loading parsed documents from pickle
